<a href="https://colab.research.google.com/github/MiriamYoussef/Library/blob/main/SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install 'git+https://github.com/facebookresearch/segment-anything.git'

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-bqbjzj_4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-bqbjzj_4
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36587 sha256=fdf2e1aaeac75687e23fed3d6ba267dff191dd047326ab61d0255afa7c5468d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-liz5t1rm/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


In [ ]:
pip install -q roboflow supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
!wget -q 'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth'


In [ ]:
!pip install ArabicOcr
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 28.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import openpyxl
from segment_anything import SamAutomaticMaskGenerator
import torch
from segment_anything import sam_model_registry
import supervision as sv
from ArabicOcr import arabicocr
from openpyxl import Workbook
import os

def extract_AkherSaa(input_string):
    # Extract the first 8 characters of the string
    title = input_string[:9].strip()
    issue = input_string[10:13].strip()
    date = input_string[14:18].strip()
    page = input_string[19:26].strip()

    return title, issue, date, page


def extract_AlArousa(input_string):
    # Extract the first 8 characters of the string
    title = input_string[:10].strip()
    issue = input_string[11:15].strip()
    date = input_string[19:22].strip()
    page = input_string[23:30].strip()
    return title, issue, date, page

def extract_AlMusawwar(input_string):
    # Extract the first 8 characters of the string
    title = input_string[:10].strip()
    issue = input_string[11:14].strip()
    date = input_string[15:19].strip()
    page = input_string[20:27].strip()

    return title, issue, date, page

def extract_AlKawakeb(input_string):
    title = input_string[:10].strip()
    issue = input_string[11:13].strip()
    date = input_string[14:18].strip()
    page = input_string[19:26].strip()
    return title, issue, date, page


folder_path ='AlAroussa-144-1927'
output_folder = 'images'

file_list = [file for file in os.listdir(folder_path) if file.lower().endswith('.jpg')]
print(len(file_list))
for file in file_list:
    print(file,"\n")

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"
sam = sam_model_registry[MODEL_TYPE](checkpoint="/content/sam_vit_h_4b8939.pth")
sam.to(device=DEVICE)
mask_generator = SamAutomaticMaskGenerator(sam)

# Create the workbook and get the active sheet

output_excel_path = output_folder+"/output.xlsx"

# Check if the workbook exists
if os.path.exists(output_excel_path):
    # Load the existing workbook
    workbook = openpyxl.load_workbook(output_excel_path)
else:
    # Create a new workbook if it doesn't exist
    workbook = openpyxl.Workbook()


sheet = workbook.active

# Save the headers for the information
sheet.cell(row=1, column=1).value = "Title"
sheet.cell(row=1, column=2).value = "Issue"
sheet.cell(row=1, column=3).value = "Date"
sheet.cell(row=1, column=4).value = "Page"
sheet.cell(row=1, column=5).value = "Arabic Text"
sheet.cell(row=1, column=6).value = "Image title"


# Save the workbook once after writing the headers
#output_excel_path = os.path.join(output_folder, "output.xlsx")
workbook.save(output_excel_path)

# Iterate through the files and process each one
for file_name in file_list:
    IMAGE_PATH = os.path.join(folder_path, file_name)
    print("File Name: ", file_name)
    title, issue, date, page = None, None, None, None

    if "Akher-Saa" in file_name:
        title, issue, date, page = extract_AkherSaa(file_name)
        print(title, issue, date, page)
    elif "Al Aroussa" in file_name:
        title, issue, date, page = extract_AlArousa(file_name)
        print(title, issue, date, page)
    elif "AlMusawwar" in file_name:
        title, issue, date, page = extract_AlMusawwar(file_name)
        print(title, issue, date, page)
    elif "AlKawakeb" in file_name:
        title, issue, date, page = extract_AlKawakeb(file_name)
        print(title, issue, date, page)

    image_bgr = cv2.imread(IMAGE_PATH)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    result = mask_generator.generate(image_rgb)

    mask_annotator = sv.MaskAnnotator(color_lookup=sv.ColorLookup.INDEX)
    detections = sv.Detections.from_sam(result)
    annotated_image = mask_annotator.annotate(image_bgr, detections)

    annotated_image_path = os.path.join(output_folder, f"annotated_image_{file_name}")
    cv2.imwrite(annotated_image_path, annotated_image)
    print(f"Annotated image saved at: {annotated_image_path}")

    # Get the cropped image from the result[0]['crop_box'] using image_bgr[x:x2, y:y2]
    sorted_list = sorted(result, key=lambda k: k['area'], reverse=True)

    i = 0
    for _, item in enumerate(sorted_list, start=0):
        crop_box = item['bbox']
        x, y, w, h = map(round, crop_box)


        # Extract the coordinates to slice the image
        try:
        # Extract the coordinates to slice the image
            cropped_image = image_bgr[y:y + h, x:x + w]

            crop_output_path = os.path.join(output_folder, f"cropped_region_{file_name}_row{i}.jpg")
            cv2.imwrite(crop_output_path, cropped_image)

        # Use the saved image file for OCR
            image_path = crop_output_path
            out_image = os.path.join(output_folder, f"out{i}.jpg")
            results = arabicocr.arabic_ocr(image_path, out_image)

            words = [result[1] for result in results]

        # Add a new row for each image to the sheet
            row_index = sheet.max_row + 1

        # Save information to the sheet (excluding image part)
            sheet.cell(row=row_index, column=1).value = title
            sheet.cell(row=row_index, column=2).value = issue
            sheet.cell(row=row_index, column=3).value = date
            sheet.cell(row=row_index, column=4).value = page
            # Add words to the sheet
            sheet.cell(row=row_index, column=5).value = ' '.join(words)
            sheet.cell(row=row_index, column=6).value = f"cropped_region_{file_name}_row{i}.jpg"

            i += 1

            if i == 5:
                break
        except TypeError as e:
            print(f"Error: {e}. Skipping this iteration.")
            continue
# Save the workbook after processing all images
    workbook.save(output_excel_path)

File Name:  Al Aroussa-1927-iss127_Page_02.jpg
Al Aroussa 1927 127 Page_02
